# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


In [3]:
# print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# get total number of rows 
print(len(full_data_rows_list))

8056


In [6]:
# check what the list of event data rows will look like
# print(full_data_rows_list)

In [7]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [8]:
df = pd.read_csv('event_datafile_new.csv', encoding = 'utf8')
df.head(6)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Sydney Youngblood,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",954,Ain't No Sunshine,73
1,Gang Starr,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",984,My Advice 2 You (Explicit),24
2,3OH!3,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",984,My First Kiss (Feat. Ke$ha) [Album Version],24
3,RÃÂ¶yksopp,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",954,The Girl and The Robot,73
4,Kajagoogoo,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",984,Too Shy,24
5,The Jimmy Castor Bunch,Layla,F,91,Griffin,306.54649,paid,"Lake Havasu City-Kingman, AZ",984,Potential,24


# Part II

#### Creating a Cluster

In [9]:
# make a connection to a Cassandra
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS alaa 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [11]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('alaa')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

- 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


- 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

- 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Design tables

**The columns we have in `event_datafile_new.csv`:**
- artist 
- firstname - of user
- gender - of user
- iteminsession - item number in session
- lastname - of user
- length - of the song
- level - (paid or free song)
- location - of the user
- sessionid
- song - title
- userid

**Our tables are:**

We created our tables based on the queries above.

#### Tbale 1:

`Table Name: artist_library
column 1: sessionid
column 2: iteminsession
column 3: artist
Column 4: song
Column 5: length
PRIMARY KEY(sessionid, iteminSession)`

**Primary Key:** sessionId , itemInSession

**Partition Key:** sessionId

**Clustering Column:** itemInSession


#### Tbale 2:

`Table Name: names_titles_library
column 1: userid
column 2: sessionid
column 3: iteminsession
Column 4: artist
Column 5: firstname
Column 6: lastname
Column 7: song
PRIMARY KEY(userid, sessionid, iteminSession)`

**Primary Key:** userid

**Partition Key:** userid

**Clustering Column:** sessionid , iteminSession


#### Tbale 3:

`Table Name: users_library
column 1: song
column 2: userid
column 3: firstname
Column 4: lastname
PRIMARY KEY(song, userid)`

**Primary Key:** song

**Partition Key:** song , userid

**Clustering Column:** userid

### Query 1:

- 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [12]:
# Createn table artist_library
query = "CREATE TABLE IF NOT EXISTS artist_library"
query = query + "(sessionid int, iteminsession int, artist text, song text, length float,\
                  PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
# Insert query for artist_library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library (sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [14]:
## Select statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM artist_library WHERE sessionid=338 AND iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2:

- 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
# Createn table names_titles_library
query1 = "CREATE TABLE IF NOT EXISTS names_titles_library"
query1 = query1 + "(userid int, sessionid int, iteminsession int, artist text, firstname text,\
                    lastname text, song text, PRIMARY KEY (userid, sessionid, iteminsession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)                  

In [16]:
# Insert query for names_titles_library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO names_titles_library (userid, sessionid, iteminsession, artist, firstname,\
                                                    lastname, song)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4],\
                                 line[9]))

In [17]:
## Select statement to verify the data was entered into the table
query1 = "SELECT artist, song, firstname, lastname\
          FROM names_titles_library\
          WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3:

- 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [18]:
# Createn table users_library 
query2 = "CREATE TABLE IF NOT EXISTS users_library"
query2 = query2 + "(song text, userid int, firstname text, lastname text, PRIMARY KEY (song, userid))"
try:
    session.execute(query2)
except Exception as e:
    print(e)            

In [19]:
# Insert query for users_library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = "INSERT INTO users_library (song, userid, firstname, lastname)"
        query2 = query2 + " VALUES (%s, %s, %s, %s)"
        session.execute(query2, (line[9], int(line[10]), line[1], line[4]))

In [20]:
## Select statement to verify the data was entered into the table
query2 = "SELECT firstname, lastname FROM users_library WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
## Drop the tables

query = "drop table artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query1 = "drop table names_titles_library"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query2 = "drop table users_library"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
# Close session and cluster connection
session.shutdown()
cluster.shutdown()